## CREATORS OF THIS FILE
Merlijn Sevenhuijsen
Alessandro Meroli


In [6]:
# Import libraries
import os
import sys
import requests
import json
import pandas as pd
from dotenv import load_dotenv


In [2]:
# Get the API key from the .env
load_dotenv()
API_KEY = os.getenv("API-KEY-TOMTOM")

In [41]:
# Dictionary of all parameters for the API
api_params = {
    'base_url': 'api.tomtom.com',
    'API_KEY': API_KEY,
    'min_lon': 18.00,
    'max_lon': 18.16,
    'min_lat': 59.25,
    'max_lat': 59.40,
    'version_number': 5,
    'category_filter': '0%2C1%2C2%2C3%2C4%2C5%2C6%2C7%2C8%2C9%2C10%2C11%2C14',
    'time_validity_filter': 'present',
    'language': 'en-GB',
    'fields': '%7Bincidents%7Btype%2Cgeometry%7Bcoordinates%7D%2Cproperties%7Bid%2CmagnitudeOfDelay%2Cevents%7Bdescription%2Ccode%2CiconCategory%7D%2CstartTime%2CendTime%7D%7D%7D'
}
#    'fields': '%7Bincidents%7Btype%2Cgeometry%7Bcoordinates%7D%2Cproperties%7Bid%2CmagnitudeOfDelay%2Cevents%7Bdescription%2Ccode%2CiconCategory%7D%2CstartTime%2CendTime%7D%7D%7D'

# Function to make  a request getting the incident details
def get_incident_details(params):
    url = f"https://{params['base_url']}/traffic/services/{params['version_number']}/incidentDetails?bbox={params['min_lon']}%2C{params['min_lat']}%2C{params['max_lon']}%2C{params['max_lat']}&fields={params['fields']}&language={params['language']}&categoryFilter={params['category_filter']}&timeValidityFilter={params['time_validity_filter']}&key={params['API_KEY']}"
    return json.loads(requests.get(url).text)
api_response = get_incident_details(api_params)
incidents = get_incident_details(api_params)['incidents']


{'incidents': [{'type': 'Feature', 'properties': {'id': '9692391e0d61244f07bfc74faf47496e', 'magnitudeOfDelay': 0, 'startTime': '2023-12-19T08:22:00Z', 'endTime': None, 'events': [{'code': 701, 'description': 'Roadworks', 'iconCategory': 9}]}, 'geometry': {'coordinates': [[18.0001853029, 59.2807983268], [18.0001303176, 59.2808291553], [17.9998942832, 59.2809511844], [17.9992679874, 59.2812770644]]}}, {'type': 'Feature', 'properties': {'id': 'db6f7faca5aa6aa3768ddbb05faee66d', 'magnitudeOfDelay': 0, 'startTime': '2023-12-18T09:33:30Z', 'endTime': None, 'events': [{'code': 701, 'description': 'Roadworks', 'iconCategory': 9}]}, 'geometry': {'coordinates': [[18.0001853029, 59.2807983268], [18.0006238441, 59.2805542241], [18.0007364969, 59.2804965487], [18.001148216, 59.2802926928], [18.001649789, 59.2800620314]]}}, {'type': 'Feature', 'properties': {'id': '751f2185a3fe8cb032f5ee1f51ee0aac', 'magnitudeOfDelay': 0, 'startTime': '2021-07-31T08:51:00Z', 'endTime': '2026-01-03T09:51:00Z', 'even

In [57]:
# Convert the response to a dataframe, we make rows from each incident

# Create a list that will contain all the incidents
list_incidents = []

# For each active incident
for i in range(len(incidents)):
    # Create a dictionary with the incident details
    dict_incident = {
        'id': incidents[i]['properties']['id'],
        'magnitudeOfDelay': incidents[i]['properties']['magnitudeOfDelay'],
        'startTime': incidents[i]['properties']['startTime'],
        'endTime': incidents[i]['properties']['endTime'],
        'type': incidents[i]['type'],
        'description': incidents[i]['properties']['events'][0]['description'],
        'code': incidents[i]['properties']['events'][0]['code'],
        'iconCategory': incidents[i]['properties']['events'][0]['iconCategory'],
        'description': incidents[i]['properties']['events'][0]['description']
    }

    # Get the coordinates of the incident, if statement is for if the incident only has one coordinate
    if any(isinstance(j, list) for j in incidents[i]['geometry']['coordinates']):
        dict_incident['lon'] = incidents[i]['geometry']['coordinates'][0][0]
        dict_incident['lat'] = incidents[i]['geometry']['coordinates'][0][1]
    else:
        dict_incident['lon'] = incidents[i]['geometry']['coordinates'][0]
        dict_incident['lat'] = incidents[i]['geometry']['coordinates'][1]

    # Add the incident to the dataframe
    list_incidents.append(dict_incident)

# Create a dataframe from the list of incidents
df_incidents = pd.DataFrame(list_incidents)

# Print the dataframe
print(df_incidents.head())

{'id': '9692391e0d61244f07bfc74faf47496e', 'magnitudeOfDelay': 0, 'startTime': '2023-12-19T08:22:00Z', 'endTime': None, 'type': 'Feature', 'description': 'Roadworks', 'code': 701, 'iconCategory': 9, 'lon': 18.0001853029, 'lat': 59.2807983268}
{'id': 'db6f7faca5aa6aa3768ddbb05faee66d', 'magnitudeOfDelay': 0, 'startTime': '2023-12-18T09:33:30Z', 'endTime': None, 'type': 'Feature', 'description': 'Roadworks', 'code': 701, 'iconCategory': 9, 'lon': 18.0001853029, 'lat': 59.2807983268}
{'id': '751f2185a3fe8cb032f5ee1f51ee0aac', 'magnitudeOfDelay': 0, 'startTime': '2021-07-31T08:51:00Z', 'endTime': '2026-01-03T09:51:00Z', 'type': 'Feature', 'description': 'Lane closed', 'code': 500, 'iconCategory': 7, 'lon': 18.0046538631, 'lat': 59.3589511601}


In [17]:
incidents

[{'type': 'Feature',
  'properties': {'iconCategory': 9},
  'geometry': {'type': 'LineString',
   'coordinates': [[18.0001853029, 59.2807983268],
    [18.0001303176, 59.2808291553],
    [17.9998942832, 59.2809511844],
    [17.9992679874, 59.2812770644]]}},
 {'type': 'Feature',
  'properties': {'iconCategory': 9},
  'geometry': {'type': 'LineString',
   'coordinates': [[18.0001853029, 59.2807983268],
    [18.0006238441, 59.2805542241],
    [18.0007364969, 59.2804965487],
    [18.001148216, 59.2802926928],
    [18.001649789, 59.2800620314]]}},
 {'type': 'Feature',
  'properties': {'iconCategory': 7},
  'geometry': {'type': 'Point',
   'coordinates': [18.0046538631, 59.3589511601]}},
 {'type': 'Feature',
  'properties': {'iconCategory': 7},
  'geometry': {'type': 'Point',
   'coordinates': [18.0046538631, 59.3589511601]}},
 {'type': 'Feature',
  'properties': {'iconCategory': 7},
  'geometry': {'type': 'Point',
   'coordinates': [18.0047316472, 59.3591214804]}},
 {'type': 'Feature',
  'pr